# Molecular Dynamics Simulation

In this example we will run an MD simulation using ASE and visualise it using ZnDraw.

In [7]:
from zndraw import ZnDraw

from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units

from ase.calculators.emt import EMT

size = 2

# Set up a crystal
atoms = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    symbol="Cu",
    size=(size, size, size),
    pbc=True,
)

# Describe the interatomic interactions with the Effective Medium Theory
atoms.calc = EMT()

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, temperature_K=300)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.


def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print(
        "Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  "
        "Etot = %.3feV" % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin)
    )

In [8]:
vis = ZnDraw(url="http://127.0.0.1:47823/", token="fcd45c3917d34b1a8329bd5a5f172c0f")

In [9]:
# Now run the dynamics

# dyn.attach(printenergy, interval=1)
dyn.attach(lambda: vis.append(atoms), interval=1)
printenergy()
dyn.run(200)

Energy per atom: Epot = -0.006eV  Ekin = 0.040eV (T=306K)  Etot = 0.034eV


True